In [ ]:
pip install prophet

In [ ]:
import pandas as pd
from fbprophet import Prophet
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
pip install fbprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 809.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet

In [ ]:
def preprocess_and_predict(df, future_period=365):
    # Preprocess data
    features = ['Open', 'High', 'Low', 'Volume', 'H-L', 'O-C', '7 DAYS MA', '14 DAYS MA', '21 DAYS MA', '7 DAYS STD DEV']
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])  # Scale the features
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    # Split the data into train and future data
    train_df = df.dropna().reset_index(drop=True)

    # Set up Prophet model
    model = Prophet(daily_seasonality=True)
    for feature in features:
        model.add_regressor(feature)

    # Fit the model on historical data
    model.fit(train_df)

    # Prepare future dataframe for prediction
    future_df = model.make_future_dataframe(periods=future_period, include_history=True)
    # Merge future features - this step assumes you have a way to estimate future values of your features
    future_df = future_df.merge(df.drop(columns='y'), on='ds', how='left')
    future_df.fillna(method='ffill', inplace=True)  # Forward fill to handle missing future values

    # Make predictions
    forecast = model.predict(future_df)

    # Plot results
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_df['ds'], y=train_df['y'], mode='lines+markers', name='Actual'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted'))
    fig.update_layout(title='Stock Price Prediction with Multiple Features', xaxis_title='Date', yaxis_title='Stock Price', hovermode='x unified')
    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
forecast_bbca = preprocess_and_predict('BBCA_clean_modified.csv')
forecast_bbni = preprocess_and_predict('BBNI_clean_modified.csv')
forecast_bbri = preprocess_and_predict('BBRI_clean_modified.csv')
forecast_bmri = preprocess_and_predict('BMRI_clean_modified.csv')

TypeError: string indices must be integers

In [ ]:
import pandas as pd
from prophet import Prophet  # Updated import statement
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

def preprocess_and_predict(csv_file, future_period=365):
    # Load data
    df = pd.read_csv(csv_file)

    # Check and print columns to ensure all are correct
    print("DataFrame columns:", df.columns)

    # Define features and ensure columns are available in DataFrame
    features = ['Open', 'High', 'Low', 'Volume', 'H-L', 'O-C', '7 DAYS MA', '14 DAYS MA', '21 DAYS MA', '7 DAYS STD DEV']
    if not all(feature in df.columns for feature in features):
        raise ValueError("One or more features are not in the DataFrame columns.")

    # Ensure 'Date' and 'Close' are in the DataFrame
    if 'Date' not in df.columns or 'Close' not in df.columns:
        raise ValueError("Necessary columns 'Date' or 'Close' are missing in DataFrame.")

    # Preprocess features
    scaler = MinMaxScaler()
    df[features] = scaler.fit_transform(df[features])  # Scale the features
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    # Split data (Optional, only if you want a specific train-test split)

    # Setup Prophet model
    model = Prophet(daily_seasonality=True)
    for feature in features:
        model.add_regressor(feature)

    # Fit the model
    model.fit(df)

    # Prepare future data for prediction
    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future.merge(df.drop(columns=['y']), on='ds', how='left').fillna(method='ffill')

    # Predict
    forecast = model.predict(future)

    # Visualization
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers', name='Actual Data'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predictions'))
    fig.update_layout(title='Stock Price Prediction', xaxis_title='Date', yaxis_title='Stock Price')
    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Example usage
try:
    prediction_results = preprocess_and_predict('BBCA_clean_modified.csv')
except Exception as e:
    print(e)

DataFrame columns: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'H-L',
       'O-C', '7 DAYS MA', '14 DAYS MA', '21 DAYS MA', '7 DAYS STD DEV'],
      dtype='object')
Found NaN in column '7 DAYS MA'


In [ ]:
def preprocess_and_predict(csv_file, future_period=365):

    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    model = Prophet(daily_seasonality=True)
    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    last_valid_date = df['ds'].max()
    future = future[future['ds'] <= last_valid_date]

    forecast = model.predict(future)
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers',
                             name='Actual Data', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines',
                             name='Predictions', line=dict(color='red')))

    fig.update_layout(
        title='Stock Price Prediction BBCA',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        hovermode='x unified',
        xaxis=dict(
            rangeslider=dict(
                visible=True
            ),
            type='date'
        ),
        yaxis=dict(
            autorange=True,
            fixedrange=False
        )
    )

    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


try:
    prediction_results = preprocess_and_predict('BBCA_clean_modified.csv')
except Exception as e:
    print("Error:", e)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/vcs7who0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/8sv9o4yq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=74719', 'data', 'file=/tmp/tmphl430r1_/vcs7who0.json', 'init=/tmp/tmphl430r1_/8sv9o4yq.json', 'output', 'file=/tmp/tmphl430r1_/prophet_model7u5u8e92/prophet_model-20240619143132.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:31:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:31:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

def preprocess_and_predict(csv_file, future_period=365):
    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    model = Prophet(
        daily_seasonality=True,
        changepoint_prior_scale=0.05,
        seasonality_mode='multiplicative',
        seasonality_prior_scale=10.0,
        holidays=None
    )

    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future[future['ds'] <= df['ds'].max()]

    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat']].merge(df[['ds', 'y']], on='ds')

    mae = mean_absolute_error(forecast_filtered['y'], forecast_filtered['yhat'])
    rmse = mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat'], squared=False)

    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers',
                             name='Actual Data', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines',
                             name='Predictions', line=dict(color='red')))

    fig.update_layout(
        title='Stock Price Prediction BBCA',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        hovermode='x unified',
        xaxis=dict(rangeslider=dict(visible=True), type='date'),
        yaxis=dict(autorange=True, fixedrange=False)
    )

    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

try:
    prediction_results = preprocess_and_predict('BBCA_clean_modified.csv')
except Exception as e:
    print("Error:", e)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/t6susgtc.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/aj85phnj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=27758', 'data', 'file=/tmp/tmphl430r1_/t6susgtc.json', 'init=/tmp/tmphl430r1_/aj85phnj.json', 'output', 'file=/tmp/tmphl430r1_/prophet_modeletqbp1pe/prophet_model-20240619144246.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:42:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:42:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Error (MAE): 83.13706535245295
Root Mean Squared Error (RMSE): 112.5798368039795


In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

def preprocess_and_predict(csv_file, future_period=365):
    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    # Setup Prophet model with custom settings
    model = Prophet(
        daily_seasonality=True,
        changepoint_prior_scale=0.05,  # Adjust the flexibility of the trend, default is 0.05
        seasonality_mode='multiplicative',  # Can be 'additive' or 'multiplicative'
        seasonality_prior_scale=10.0,  # Prior scale for seasonality component
        holidays=None  # Define holiday dataframe if relevant
    )

    # Assuming any additional regressors would be added here, e.g., model.add_regressor('feature')

    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future[future['ds'] <= df['ds'].max()]

    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat']].merge(df[['ds', 'y']], on='ds')

    mae = mean_absolute_error(forecast_filtered['y'], forecast_filtered['yhat'])
    rmse = mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat'], squared=False)

    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers', name='Actual Data', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predictions', line=dict(color='red')))

    fig.update_layout(
        title='Stock Price Prediction BBRI',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        hovermode='x unified',
        xaxis=dict(rangeslider=dict(visible=True), type='date'),
        yaxis=dict(autorange=True, fixedrange=False)
    )

    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Example usage
try:
    prediction_results = preprocess_and_predict('BBRI_clean_modified.csv')
except Exception as e:
    print("Error:", e)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/1ll_14sb.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/nv5l9jue.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=25051', 'data', 'file=/tmp/tmphl430r1_/1ll_14sb.json', 'init=/tmp/tmphl430r1_/nv5l9jue.json', 'output', 'file=/tmp/tmphl430r1_/prophet_model910swlt7/prophet_model-20240619144235.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:42:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:42:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Error (MAE): 112.40018393903908
Root Mean Squared Error (RMSE): 149.0212966135785


In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

def preprocess_and_predict(csv_file, future_period=365):
    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    # Setup Prophet model with custom settings
    model = Prophet(
        daily_seasonality=True,
        changepoint_prior_scale=0.05,  # Adjust the flexibility of the trend, default is 0.05
        seasonality_mode='multiplicative',  # Can be 'additive' or 'multiplicative'
        seasonality_prior_scale=10.0,  # Prior scale for seasonality component
        holidays=None  # Define holiday dataframe if relevant
    )

    # Assuming any additional regressors would be added here, e.g., model.add_regressor('feature')

    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future[future['ds'] <= df['ds'].max()]

    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat']].merge(df[['ds', 'y']], on='ds')

    mae = mean_absolute_error(forecast_filtered['y'], forecast_filtered['yhat'])
    rmse = mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat'], squared=False)

    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers', name='Actual Data', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predictions', line=dict(color='red')))

    fig.update_layout(
        title='Stock Price Prediction BMRI',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        hovermode='x unified',
        xaxis=dict(rangeslider=dict(visible=True), type='date'),
        yaxis=dict(autorange=True, fixedrange=False)
    )

    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Example usage
try:
    prediction_results = preprocess_and_predict('BMRI_clean_modified.csv')
except Exception as e:
    print("Error:", e)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/k0l0dvo7.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/08r8_wku.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8232', 'data', 'file=/tmp/tmphl430r1_/k0l0dvo7.json', 'init=/tmp/tmphl430r1_/08r8_wku.json', 'output', 'file=/tmp/tmphl430r1_/prophet_model9z8y4zj2/prophet_model-20240619144324.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:43:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:43:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Error (MAE): 124.8390427506149
Root Mean Squared Error (RMSE): 165.93821571193217


In [ ]:
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

def preprocess_and_predict(csv_file, future_period=365):
    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    # Setup Prophet model with custom settings
    model = Prophet(
        daily_seasonality=True,
        changepoint_prior_scale=0.05,  # Adjust the flexibility of the trend, default is 0.05
        seasonality_mode='multiplicative',  # Can be 'additive' or 'multiplicative'
        seasonality_prior_scale=10.0,  # Prior scale for seasonality component
        holidays=None  # Define holiday dataframe if relevant
    )

    # Assuming any additional regressors would be added here, e.g., model.add_regressor('feature')

    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future[future['ds'] <= df['ds'].max()]

    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat']].merge(df[['ds', 'y']], on='ds')

    mae = mean_absolute_error(forecast_filtered['y'], forecast_filtered['yhat'])
    rmse = mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat'], squared=False)

    print(f'Mean Absolute Error (MAE): {mae}')
    print(f'Root Mean Squared Error (RMSE): {rmse}')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines+markers', name='Actual Data', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predictions', line=dict(color='red')))

    fig.update_layout(
        title='Stock Price Prediction BBNI',
        xaxis_title='Date',
        yaxis_title='Stock Price',
        hovermode='x unified',
        xaxis=dict(rangeslider=dict(visible=True), type='date'),
        yaxis=dict(autorange=True, fixedrange=False)
    )

    fig.show()

    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

# Example usage
try:
    prediction_results = preprocess_and_predict('BBNI_clean_modified.csv')
except Exception as e:
    print("Error:", e)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/e9vwo1jq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/btj4gs8u.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=68148', 'data', 'file=/tmp/tmphl430r1_/e9vwo1jq.json', 'init=/tmp/tmphl430r1_/btj4gs8u.json', 'output', 'file=/tmp/tmphl430r1_/prophet_modelfkzuzza8/prophet_model-20240619145349.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:53:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:53:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Mean Absolute Error (MAE): 99.4591642802698
Root Mean Squared Error (RMSE): 127.70852014506293


In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

def preprocess_and_predict(csv_file, future_period=365):
    df = pd.read_csv(csv_file)
    df['ds'] = pd.to_datetime(df['Date'])
    df['y'] = df['Close']

    model = Prophet(daily_seasonality=True, changepoint_prior_scale=0.05, seasonality_mode='multiplicative')
    model.fit(df)

    future = model.make_future_dataframe(periods=future_period, include_history=True)
    future = future[future['ds'] <= df['ds'].max()]  # Only predict up to the last known data point

    forecast = model.predict(future)
    forecast_filtered = forecast[['ds', 'yhat']].merge(df[['ds', 'y']], on='ds')

    mae = mean_absolute_error(forecast_filtered['y'], forecast_filtered['yhat'])
    rmse = mean_squared_error(forecast_filtered['y'], forecast_filtered['yhat'], squared=False)

    return mae, rmse

# List to hold results for multiple datasets
results = []

# Datasets to process
datasets = ['BBCA_clean_modified.csv', 'BBNI_clean_modified.csv', 'BBRI_clean_modified.csv', 'BMRI_clean_modified.csv']
dataset_names = ['BBCA', 'BBNI', 'BBRI', 'BMRI']

# Process each dataset
for csv_file, name in zip(datasets, dataset_names):
    mae, rmse = preprocess_and_predict(csv_file)
    results.append({
        'Dataset': name,
        'MAE': mae,
        'RMSE': rmse
    })

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
print(results_df)

DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/uzxvsui1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/qgl_0l7i.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=29494', 'data', 'file=/tmp/tmphl430r1_/uzxvsui1.json', 'init=/tmp/tmphl430r1_/qgl_0l7i.json', 'output', 'file=/tmp/tmphl430r1_/prophet_model1qu7_18h/prophet_model-20240619144008.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:40:08 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:40:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/0et5wj6d.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphl430r1_/9xaq1c9o.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

  Dataset         MAE        RMSE
0    BBCA   83.137065  112.579837
1    BBNI   99.459164  127.708520
2    BBRI  112.400184  149.021297
3    BMRI  124.839043  165.938216
